Stack Overflow sample database obtained from https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/.

Original source: https://archive.org/details/stackexchange issued under license cc-by-sa 3.0 license (https://creativecommons.org/licenses/by-sa/3.0/)

## Demo 1 - Managed Instance Restore from Azure Blob Storage

Assumptions
* Azure blob storage has been provisioned.
* Source SQL Server database has been backed up to Azure blob storage containers.

For more information on how to migrate databases to Managed Instance via Backup to URL please see [here](https://www.mssqltips.com/sqlservertip/5457/migrating-sql-server-databases-to-azure-managed-instances-with-native-backups/)

### Step 1 - Create Credentails to access blob storage

In [1]:
CREATE CREDENTIAL [https://<StorageAccount>.blob.core.windows.net/stackoverflowdualdatafile]
    WITH
        IDENTITY='Shared Access Signature',
        SECRET='<SAS Key>'
;
GO
CREATE CREDENTIAL [https://<StorageAccount>.blob.core.windows.net/stackoverflowsingledatafile]
    WITH
        IDENTITY='Shared Access Signature',
        SECRET='<SAS Key>'
;
GO
CREATE CREDENTIAL [https://<StorageAccount>.blob.core.windows.net/stackoverflowquaddatafile]
    WITH
        IDENTITY='Shared Access Signature',
        SECRET='<SAS Key>'
;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.203

### Step 2 - Restore Database to Managed Instance

In [2]:
RESTORE DATABASE StackOverflow2013_SingleFile
    FROM URL = N'https://<StorageAccount>.blob.core.windows.net/stackoverflowsingledatafile/stackoverflow2013single.bak'
;
GO

Commands completed successfully.

Total execution time: 00:27:45.740

In [0]:
RESTORE DATABASE StackOverflow2013_DualFile
    FROM URL = N'https://<StorageAccount>.blob.core.windows.net/stackoverflowdualdatafile/stackoverflow2013dual.bak'
;
GO

In [3]:
RESTORE DATABASE StackOverflow2013_QuadFile
    FROM URL = N'https://<StorageAccount>.blob.core.windows.net/stackoverflowquaddatafile/stackoverflow2013quad.bak'
;
GO

Commands completed successfully.

Total execution time: 00:27:05.792

## Demo 2 - Query Performance
This demo will look at how the number of files and size of files will impact query performance. Query details;
* Query Source: https://data.stackexchange.com/stackoverflow/query/952/top-500-answerers-on-the-site
* Query Author: Sam Saffron (https://data.stackexchange.com/users/1/sam-saffron)

### Step 1 - Baesline performance query

In [4]:
USE [StackOverflow2013_SingleFile];
GO

SELECT 
    TOP 500
    Users.Id as [User Link],
    Count(Posts.Id) AS Answers,
    CAST(AVG(CAST(Score AS float)) as numeric(6,2)) AS [Average Answer Score]
FROM
    Posts
  INNER JOIN
    Users ON Users.Id = OwnerUserId
WHERE 
    PostTypeId = 2 and CommunityOwnedDate is null and ClosedDate is null
GROUP BY
    Users.Id, DisplayName
HAVING
    Count(Posts.Id) > 10
ORDER BY
    [Average Answer Score] DESC

Commands completed successfully.

: Query failed: The operation was canceled.

Query was canceled by user

Total execution time: 00:03:09.301

### Step 2 - Increase number of files (two data files)

In [0]:
USE [StackOverflow2013_Dual];
GO

SELECT 
    TOP 500
    Users.Id as [User Link],
    Count(Posts.Id) AS Answers,
    CAST(AVG(CAST(Score AS float)) as numeric(6,2)) AS [Average Answer Score]
FROM
    Posts
  INNER JOIN
    Users ON Users.Id = OwnerUserId
WHERE 
    PostTypeId = 2 and CommunityOwnedDate is null and ClosedDate is null
GROUP BY
    Users.Id, DisplayName
HAVING
    Count(Posts.Id) > 10
ORDER BY
    [Average Answer Score] DESC

### Step 3 - Increase number of files again (four)

In [0]:
USE [StackOverflow2013_Quad];
GO

SELECT 
    TOP 500
    Users.Id as [User Link],
    Count(Posts.Id) AS Answers,
    CAST(AVG(CAST(Score AS float)) as numeric(6,2)) AS [Average Answer Score]
FROM
    Posts
  INNER JOIN
    Users ON Users.Id = OwnerUserId
WHERE 
    PostTypeId = 2 and CommunityOwnedDate is null and ClosedDate is null
GROUP BY
    Users.Id, DisplayName
HAVING
    Count(Posts.Id) > 10
ORDER BY
    [Average Answer Score] DESC

### Step 4 - Increase file size to increase performance

In [0]:
ALTER DATABASE StackOverflow2013_SingleFile
    MODIFY FILE
    (
        NAME = '<FileName>',
        size = 1TB
    )
;
GO

### Step 5 - Re-run baseline query on single file db.

In [0]:
USE [StackOverflow2013_SingleFile];
GO

SELECT 
    TOP 500
    Users.Id as [User Link],
    Count(Posts.Id) AS Answers,
    CAST(AVG(CAST(Score AS float)) as numeric(6,2)) AS [Average Answer Score]
FROM
    Posts
  INNER JOIN
    Users ON Users.Id = OwnerUserId
WHERE 
    PostTypeId = 2 and CommunityOwnedDate is null and ClosedDate is null
GROUP BY
    Users.Id, DisplayName
HAVING
    Count(Posts.Id) > 10
ORDER BY
    [Average Answer Score] DESC